<a href="https://colab.research.google.com/github/olgasem10/Compling-tasks/blob/master/Keywords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymorphy2

     |████████████████████████████████| 51kB 927kB/s 
     |████████████████████████████████| 7.1MB 3.3MB/s 


In [2]:
import json, os
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
%cd /content/drive/My Drive/Keys

/content/drive/My Drive/Keys


In [11]:
data = pd.read_json('All_data_norm.json', orient = 'split')
data.shape

(17266, 7)

In [0]:
pd.set_option('display.max_colwidth', 300)

In [15]:
data.head(2)

,keywords,title,content,low_keywords,content_norm,title_norm
0,"[КРАЕОБМЕТОЧНЫЕ ШВЕЙНЫЕ МАШИНЫ, РАСЧЕТ НИТЕПОДАЧИ, ДИАГРАММЫ ПОДАЧИ И ПОТРЕБЛЕНИЯ НИТИ]",Математическая модель рабочего процесса образования стежка на швейной машине,"УДК 687.053\n\nМАТЕМАТИЧЕСКАЯ МОДЕЛЬ РАБОЧЕГО ПРОЦЕССА ОБРАЗОВАНИЯ\nСТЕЖКА НА ШВЕЙНОЙ МАШИНЕ\nЕрмаков А.С.,\nкандидат технических наук, доцент,\nФГОУВПО «Московский государственный университет туризма и сервиса», г. Москва.\nThe method of device design for thread feeding of the upper looper and ...","[краеобметочные швейные машины, расчет нитеподачи, диаграммы подачи и потребления нити]","[модель, рабочий, процесс, образование, стежок, машина, ермаков, кандидат, наука, доцент, университет, туризм, сервис, год, москва, метод, проектирование, устройство, подача, нитка, петлитель, определение, процесс, потребление, основание, использование, диаграмма, согласование, функция, подача, ...","[модель, рабочий, процесс, образование, стежок, машина]"
1,"[панель из ячеистого бетона, аварийное состояние, ВЛАЖНОСТЬ, физический износ, Ремонт бетона, инженерная безопасность, cellular concrete panels, emergency condition, moisture, physical wear-out, repair concrete, engineering safety]",Инженерная безопасность эксплуатации жилых зданий со стеновыми панелями из автоклавного газобетона,"/84\n\nCivil SecurityTechnology, Vol. 8, 2011, No. 3 (29)\n\nУДК 699.88 + 692.232.44\n\nИнженерная безопасность эксплуатации жилых\nзданий со стеновыми панелями из автоклавного\nгазобетона\nISSN 1996-8493\n© Технологии гражданской безопасности, 2011\n\nВ.Б. Cальников, В.А. Беляков\nАннотация\nВ ...","[панель из ячеистого бетона, аварийное состояние, влажность, физический износ, ремонт бетона, инженерная безопасность, cellular concrete panels, emergency condition, moisture, physical wear-out, repair concrete, engineering safety]","[безопасность, эксплуатация, здание, панель, газобетон, технология, безопасность, cальник, беляков, аннотация, статья, метод, оценка, износ, панель, лёгкое, бетон, технология, безопасность, настоящее, время, печать, чаща, статья, авария, обрушение, конструкция, дом, фонд, способность, отсутствие...","[безопасность, эксплуатация, здание, панель, газобетон]"


In [0]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 5))
    print('Recall - ', round(np.mean(recalls), 5))
    print('F1 - ', round(np.mean(f1s), 5))
    print('Jaccard - ', round(np.mean(jaccards), 5))

Пересчитываем tf-idf на всех данных

In [0]:
data['content_norm_str'] = data['content_norm'].apply(' '.join)

In [0]:
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)
tfidf.fit(data['content_norm_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=5, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [0]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
texts_vectors = tfidf.transform(data['content_norm_str'])

In [0]:
all_keywords = []
for i in range(data.shape[0]):
    k_words = [id2word[w] for w in texts_vectors[i].toarray().argsort()[:,:-11:-1].tolist()[0]]
    all_keywords.append(k_words)

In [0]:
evaluate(data['low_keywords'], all_keywords)

Precision -  0.098
Recall -  0.136
F1 -  0.106
Jaccard -  0.061


Превзойти бейзлайн получилось не очень значительно, поэтому оставляю и неудачные попытки тоже

## Попытка 1

С помощью tf-idf векторизации мы получаем среди ключевых слов биграммы, но слова в них не согласованы, поэтому они не дают прироста качества. Попробуем ставить второе слово в биграмме в родительный падеж

In [0]:
def matching(bigr):
    if ' ' in bigr:
        try:
            words = bigr.split()
            parsed = [morph.parse(word)[0] for word in words]
            first = parsed[0].word
            second = parsed[1].inflect({'gent'}).word
            strng = '{} {}'.format(first, second)
        except:
            strng = bigr
    else:
        strng = bigr
    return strng

In [0]:
conc_keywords = [[matching(line) for line in all_keywords[i]] for i in range(len(all_keywords))]

In [0]:
evaluate(data['low_keywords'], conc_keywords)

Precision -  0.097
Recall -  0.137
F1 -  0.106
Jaccard -  0.06


Значения практически не изменились

## Попытка 2

Многие списки ключевых слов содержат помимо существительных прилагательные. Попробуем оставлять при нормализации прилагательные и если они окажутся среди биграм, согласовывать их с существительными по роду

In [16]:
data = pd.read_json('With_adj.json', orient = 'split')
data.shape

(17266, 7)

In [19]:
data.head(2)

,keywords,title,content,low_keywords,content_norm,title_norm
0,"[КРАЕОБМЕТОЧНЫЕ ШВЕЙНЫЕ МАШИНЫ, РАСЧЕТ НИТЕПОДАЧИ, ДИАГРАММЫ ПОДАЧИ И ПОТРЕБЛЕНИЯ НИТИ]",Математическая модель рабочего процесса образования стежка на швейной машине,"УДК 687.053\n\nМАТЕМАТИЧЕСКАЯ МОДЕЛЬ РАБОЧЕГО ПРОЦЕССА ОБРАЗОВАНИЯ\nСТЕЖКА НА ШВЕЙНОЙ МАШИНЕ\nЕрмаков А.С.,\nкандидат технических наук, доцент,\nФГОУВПО «Московский государственный университет туризма и сервиса», г. Москва.\nThe method of device design for thread feeding of the upper looper and ...","[краеобметочные швейные машины, расчет нитеподачи, диаграммы подачи и потребления нити]","[математический, модель, рабочий, процесс, образование, стежок, швейный, машина, ермаков, кандидат, технический, наука, доцент, московский, государственный, университет, туризм, сервис, год, москва, метод, проектирование, устройство, подача, нитка, верхний, петлитель, определение, рациональный, ...","[математический, модель, рабочий, процесс, образование, стежок, швейный, машина]"
1,"[панель из ячеистого бетона, аварийное состояние, ВЛАЖНОСТЬ, физический износ, Ремонт бетона, инженерная безопасность, cellular concrete panels, emergency condition, moisture, physical wear-out, repair concrete, engineering safety]",Инженерная безопасность эксплуатации жилых зданий со стеновыми панелями из автоклавного газобетона,"/84\n\nCivil SecurityTechnology, Vol. 8, 2011, No. 3 (29)\n\nУДК 699.88 + 692.232.44\n\nИнженерная безопасность эксплуатации жилых\nзданий со стеновыми панелями из автоклавного\nгазобетона\nISSN 1996-8493\n© Технологии гражданской безопасности, 2011\n\nВ.Б. Cальников, В.А. Беляков\nАннотация\nВ ...","[панель из ячеистого бетона, аварийное состояние, влажность, физический износ, ремонт бетона, инженерная безопасность, cellular concrete panels, emergency condition, moisture, physical wear-out, repair concrete, engineering safety]","[инженерный, безопасность, эксплуатация, жилой, здание, стеновый, панель, автоклавный, газобетон, технология, гражданский, безопасность, cальник, беляков, аннотация, статья, исследовательский, метод, оценка, физический, износ, панель, лёгкое, бетон, технология, гражданский, безопасность, настоящ...","[инженерный, безопасность, эксплуатация, жилой, здание, стеновый, панель, автоклавный, газобетон]"


In [0]:
data['content_norm_str'] = data['content_norm'].apply(' '.join)

In [0]:
def matching(bigr):
    if ' ' in bigr:
        try:
            words = bigr.split()
            parsed = [morph.parse(word)[0] for word in words]
            if parsed[0].tag.POS == 'ADJF':
                first = parsed[0].inflect({parsed[1].tag.gender, 'nomn'}).word
                second = parsed[1].word
                strng = '{} {}'.format(first, second)
            elif parsed[0].tag.POS == 'NOUN' and parsed[1].tag.POS == 'NOUN':
                first = parsed[0].word
                second = parsed[1].inflect({'gent'}).word
                strng = '{} {}'.format(first, second)
            else:
                strng = bigr
        except:
            strng = bigr
    else:
        strng = bigr
    return strng

In [0]:
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)
tfidf.fit(data['content_norm_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=5, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [0]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
texts_vectors = tfidf.transform(data['content_norm_str'])

In [0]:
all_keywords = []
for i in range(data.shape[0]):
    k_words = [id2word[w] for w in texts_vectors[i].toarray().argsort()[:,:-11:-1].tolist()[0]]
    all_keywords.append(k_words)

In [0]:
conc_keywords = [[matching(line) for line in all_keywords[i]] for i in range(len(all_keywords))]

In [0]:
evaluate(data['low_keywords'], conc_keywords)

Precision -  0.096
Recall -  0.139
F1 -  0.106
Jaccard -  0.06


Немного увеличился только recall

## Попытка 3

Когда мы добавили прилагательные, они оказались не только среди биграмм, но и в виде отдельных слов. Попробуем их удалить

In [0]:
all_keywords = []
for i in range(data.shape[0]):
    k_words = [id2word[w] for w in texts_vectors[i].toarray().argsort()[:,:-21:-1].tolist()[0]]
    k_words = [word for word in k_words if not morph.parse(word)[0].tag.POS == 'ADJF'][:10]
    all_keywords.append(k_words)

In [0]:
all_keywords[67]

['информационный культура',
 'культура',
 'знание',
 'субъект',
 'свобода',
 'знание информация',
 'модернизм',
 'позитивный свобода',
 'эпистемология',
 'истина']

In [0]:
conc_keywords = [[matching(line) for line in all_keywords[i]] for i in range(len(all_keywords))]

In [0]:
evaluate(data['low_keywords'], conc_keywords)

Precision -  0.104
Recall -  0.15
F1 -  0.114
Jaccard -  0.065


Получаем некоторое улучшение, хотя не очень значительное


## Попытка 4

Собрать фразы генсимом и посчитать их каунтером


In [0]:
from gensim.models.phrases import Phrases, Phraser

In [0]:
def phr(words):
  phrases = Phrases([words], min_count=4, threshold=6)
  bigram = Phraser(phrases)
  joined = bigram[words]
  joined = [word.replace('_', ' ')for word in joined]
  return joined

In [0]:
data['phrases'] = data['content_norm'].apply(phr)

In [0]:
data.to_json('With_adj_and_phrases.json', orient = 'split')

In [0]:
key_words = data['phrases'].apply(lambda x: [x[0] for x in Counter(x).most_common(20)])
k_words = [[word for word in keyword if not morph.parse(word)[0].tag.POS == 'ADJF'][:10] for keyword in key_words]
conc_keywords = [[matching(line) for line in k_words[i]] for i in range(len(k_words))]

In [0]:
conc_keywords[89]

['предприятие',
 'услуга',
 'модернизация',
 'качество',
 'стратегическая устойчивость',
 'гостиничная услуга',
 'конкурентоспособность',
 'организация',
 'устойчивость',
 'условие']

In [0]:
evaluate(data['low_keywords'], conc_keywords)

Precision -  0.114
Recall -  0.16
F1 -  0.125
Jaccard -  0.072


## Попытка 5

Поставим тот же текст с биграммами в граф

In [0]:
from itertools import combinations

In [0]:
import networkx as nx

In [0]:
def build_matrix(text, window_size=5):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    return m, id2word

def some_centrality_measure(text, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_array(matrix)
    # тут можно поставить любую метрику
    #
    node2measure = dict(nx.pagerank(G))
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [0]:
%%time
keyword_nx = data['phrases'].apply(lambda x: some_centrality_measure(x, 10, 20))

CPU times: user 34min 55s, sys: 4.09 s, total: 34min 59s
Wall time: 35min 2s


In [0]:
conc_keywords[222]

['модель',
 'грунт',
 'уравнение',
 'решение',
 'анализ',
 'классификация',
 'атрибут',
 'изменение знака',
 'разработка',
 'зона']

In [0]:
k_words = [[word for word in keyword if not morph.parse(word)[0].tag.POS == 'ADJF'][:10] for keyword in keyword_nx]
conc_keywords = [[matching(line) for line in k_words[i]] for i in range(len(k_words))]

In [0]:
evaluate(data['low_keywords'], conc_keywords)

Precision -  0.112
Recall -  0.157
F1 -  0.122
Jaccard -  0.07


## Попытка 6

Попробуем готовый пакет summa для саммаризации и извлечения ключевых слов

In [0]:
pip install summa

     |████████████████████████████████| 61kB 1.9MB/s 
  Created wheel for summa: filename=summa-1.2.0-cp36-none-any.whl size=54411 sha256=06116d5a3e56dd04b72de1e95e7dbc88e73a713d9cfb4c136d34a67b406969e6
  Stored in directory: /root/.cache/pip/wheels/6a/09/68/e2f2861c01d86407c3fa5220826ed7eed2abaa56b001be5970
Successfully built summa


In [0]:
from summa import keywords

In [0]:
from tqdm import tqdm

In [0]:
summa_keywords = []
for text in tqdm(data['content_norm_str']):
  try:
    kwords = keywords.keywords(text, words=20, language='russian').split('\n')
    
  except:
    kwords = ['']
  summa_keywords.append(kwords)

100%|██████████| 17266/17266 [2:07:58<00:00,  2.25it/s]


In [0]:
summa_keywords[23]

['метод',
 'погрешность',
 'рис',
 'время измерение',
 'напряжение',
 'определение параметр',
 'цепь мгновенный значение',
 'определённый',
 'который',
 'сопротивление',
 'выражение',
 'образцовый',
 'мелентьев',
 'технический',
 'сигнал',
 'индуктивность',
 'отношение']

Удалим отдельные прилагательные

In [0]:
s_words = [[word for word in keyword if not morph.parse(word)[0].tag.POS == 'ADJF'] for keyword in summa_keywords]

Согласуем словосочетания

In [0]:
t_words = [[matching(line) for line in s_words[i]][:10] for i in range(len(s_words))]

In [0]:
evaluate(data['low_keywords'], t_words)

Precision -  0.072
Recall -  0.098
F1 -  0.077
Jaccard -  0.043


Получилось хуже, чем бейзлайн(

## Попытка 7

Попробуем другую метрику для определения центральности

In [0]:
def other_centrality_measure(text, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_array(matrix)
    # тут можно поставить любую метрику
    #
    node2measure = dict(nx.load_centrality(G))
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [0]:
%%time
keyword_nx = data['phrases'][:10].apply(lambda x: some_centrality_measure(x, 10, 20))

CPU times: user 2.77 s, sys: 24.9 ms, total: 2.79 s
Wall time: 2.8 s


In [0]:
%%time
keyword_nx2 = data['phrases'].apply(lambda x: other_centrality_measure(x, 10, 20))

CPU times: user 3h 12min 4s, sys: 3.94 s, total: 3h 12min 8s
Wall time: 3h 12min 32s


In [0]:
k_words = [[word for word in keyword if not morph.parse(word)[0].tag.POS == 'ADJF'][:10] for keyword in keyword_nx2]
conc_keywords = [[matching(line) for line in k_words[i]] for i in range(len(k_words))]

In [0]:
evaluate(data['low_keywords'], conc_keywords)

Precision -  0.10592
Recall -  0.14832
F1 -  0.11536
Jaccard -  0.0659


Больше чем бейзлайн, но меньше чем мы получали при другой метрике

## Попытка 8
Попробуем обычный counter на существительных

In [13]:
evaluate(data['low_keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.10751
Recall -  0.14691
F1 -  0.11609
Jaccard -  0.06686
